In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/suhack2022data/restaurant-2-products-price.csv
/kaggle/input/suhack2022data/UK Holidays.xlsx
/kaggle/input/suhack2022data/london_weather.csv
/kaggle/input/suhack2022data/indian_holidays.csv
/kaggle/input/suhack2022data/restaurant-1-orders.csv
/kaggle/input/suhack2022data/restaurant-2-orders.csv
/kaggle/input/suhack2022data/restaurant-1-products-price.csv


In [2]:
rest1dat = pd.read_csv("../input/suhack2022data/restaurant-1-products-price.csv")
rest1price = pd.read_csv("../input/suhack2022data/restaurant-1-orders.csv")
rest2dat = pd.read_csv("../input/suhack2022data/restaurant-2-orders.csv")
rest2price = pd.read_csv("../input/suhack2022data/restaurant-2-products-price.csv")

weatherdat = pd.read_csv("../input/suhack2022data/london_weather.csv")
ukholidat = pd.read_excel("../input/suhack2022data/UK Holidays.xlsx")
indholidat = pd.read_csv("../input/suhack2022data/indian_holidays.csv")
print(rest1dat.isnull().sum().any())
print(rest2dat.isnull().sum().any())

False
False


In [3]:
# Predict weekly demand based on week time step, weather information and holidays (seasons)

# Testing